# Autores colombianos por provincias

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [2]:
%%time

autores_df = pd.read_csv(
    'autores_colombianos_2.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 242 ms, sys: 12.3 ms, total: 254 ms
Wall time: 253 ms


In [3]:
autores_df.shape

(5551, 13)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO,NOMBRE_ENCONTRADO
0,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bahía Solano,Chocó Department,True,Oscar Collazos
1,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,
2,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,
3,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,
4,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,


In [57]:
autores_df[autores_df['PAIS_AUTOR'] == 'DESCONOCIDO']

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO,NOMBRE_ENCONTRADO


In [58]:
autores_df[autores_df['CIUDAD_AUTOR'] == 'DESCONOCIDO'].head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO,NOMBRE_ENCONTRADO
1,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,
2,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,
3,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,
4,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,
5,Secretos de el dorado : Colombia,Colombia,Germán Arciniegas,Colombia,1990,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False,


## Departamentos Colombia

In [5]:
deps1_df = pd.read_csv('deps1.csv', sep='\t', header=-1)

In [6]:
deps1_df.shape

(33, 8)

In [7]:
deps1_df.head()

,0,1,2,3,4,5,6,7
0,0,Capital District,Enrique Peñalosa,CR,Bogotá,"1,587","8,254,722",1538
1,1,Amazonas,Manuel Antonio Carebilla Cuéllar,CR,Leticia,"109,665","80,360",1991
2,2,Antioquia,Luis Pérez,Liberal,Medellín,"63,612","5,750,478",1826
3,3,Arauca,Ricardo Alvarado Bestene,La U,Arauca,"23,818","282,302",1991
4,4,Atlántico,Eduardo I. Verano de la Rosa,Liberal,Barranquilla,"3,388","2,365,663",1910


In [8]:
deps1_df[1] = deps1_df.apply(lambda x: x[1].strip(), axis=1)

In [9]:
deps1_set = set(deps1_df[1])

In [10]:
deps2_df = pd.read_csv('deps2.txt', header=-1)

In [11]:
deps2_df.shape

(33, 1)

In [12]:
deps2_df.head()

,0
0,Antioquia
1,Caldas
2,Bogotá D.C.
3,Bolívar
4,Córdoba


In [13]:
deps2_df[0] = deps2_df.apply(lambda x: x[0].strip(), axis=1)

In [14]:
deps2_set = set(deps2_df[0])

In [15]:
deps1_set.union(deps2_set).difference(deps1_set.intersection(deps2_set))

{'Bogotá D.C.', 'Capital District'}

In [16]:
deps_mappings = {}
for d in deps1_set:
    deps_mappings[d] = d
deps_mappings['Capital District'] = 'Bogotá D.C.'

## Ciudades y departamentos

In [17]:
dtypes = {
    'Ciudad': str,
    'Población': np.int32,
    'Departamento': str,
}

ciudades_df = pd.read_csv(
    'ciudades_colombia.csv',
    sep='\t',
    header=-1,
    names=['Wiki index', 'Ciudad', 'Población', 'Departamento'],
    usecols=['Ciudad', 'Población', 'Departamento'],
    dtype=dtypes,
    thousands='.',
)

In [18]:
departamentos_sr = pd.Series(list(ciudades_df['Departamento']), index=ciudades_df['Ciudad'])

## Corrigiendo departamentos

In [27]:
print(sorted(departamentos_sr.unique()))

['Amazonas', 'Antioquia', 'Arauca', 'Archipiélago de San Andrés', 'Atlántico', 'Bolívar', 'Boyacá', 'Caldas', 'Caquetá', 'Casanare', 'Cauca', 'Cesar', 'Chocó', 'Cundinamarca', 'Córdoba', 'Guainía', 'Guaviare', 'Huila', 'La Guajira', 'Magdalena', 'Meta', 'Nariño', 'Norte de Santander', 'Putumayo', 'Quindío', 'Risaralda', 'Santander', 'Sucre', 'Tolima', 'Valle del Cauca', 'Vaupés', 'Vichada']


In [28]:
print(sorted(autores_df['DEPARTAMENTO_AUTOR'].unique()))

['Antioquia', 'Antioquia Department', 'Arauca Department', 'Atlántico', 'Atlántico (Colombia)', 'Atlántico Department', 'Bolívar', 'Boyacá', 'Boyacá Department', 'Caldas', 'Casanare', 'Cauca', 'Cauca Department', 'Cesar', 'Chocó', 'Chocó Department', 'Cundinamarca', 'Cundinamarca Department', 'Córdoba', 'DESCONOCIDO', 'Huila', 'Huila Department', 'La Guajira', 'Magdalena', 'Magdalena Department', 'Meta', 'Nariño', 'Nariño Department', 'Norte de Santander', 'Norte de Santander Department', 'Quindío', 'Quindío Department', 'Risaralda', 'Santander', 'Santander Department', 'Sucre', 'Tolima', 'Valle del Cauca', 'Valle del Cauca Department']


In [29]:
deps_mappings['Antioquia Department'] = 'Antioquia'
deps_mappings['Arauca Department'] = 'Arauca'
deps_mappings['Atlántico (Colombia)'] = 'Atlántico'
deps_mappings['Atlántico Department'] = 'Atlántico'
deps_mappings['Boyacá Department'] = 'Boyacá'
deps_mappings['Cauca Department'] = 'Cauca'
deps_mappings['Chocó Department'] = 'Chocó'
deps_mappings['Cundinamarca Department'] = 'Cundinamarca'
deps_mappings['Huila Department'] = 'Huila'
deps_mappings['Magdalena Department'] = 'Magdalena'
deps_mappings['Nariño Department'] = 'Nariño'
deps_mappings['Norte de Santander Department'] = 'Norte de Santander'
deps_mappings['Quindío Department'] = 'Quindío'
deps_mappings['Santander Department'] = 'Santander'
deps_mappings['Valle del Cauca Department'] = 'Valle del Cauca'

In [56]:
for i, x in autores_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    departamento1 = x['DEPARTAMENTO_AUTOR']
    if ciudad not in departamentos_sr or departamento1 not in deps_mappings:
        print(1, ciudad, '-', departamento1)
    else:
        departamento2 = deps_mappings[departamento1]
        departamento3 = departamentos_sr[ciudad]
        if isinstance(departamento3, str):
            if departamento2 != departamento3:
                print(2, ciudad, '-', departamento2, '-', departamento3)
        else:
            if departamento2 not in departamento3.values:
                print(3, ciudad, '-', departamento2, '-', departamento3.values)

DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
La Tebaida, Quindío - Quindío Department
La Tebaida, Quindío - Quindío Department
La Tebaida, Quindío - Quindío Department
La Tebaida, Quindío - Quindío Department
La Tebaida, Quindío - Quindío Department
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
La Tebaida, Quindío - Quindío Department
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
Cartagena de Indias - Bolívar
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
Cartagena de Indias - Bolívar
Donmatías - Antioquia
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
DESCONOCIDO - DESCONOCIDO
Cartagena de Indias - 

In [38]:
departamento2

'Quindío'

In [51]:
departamento3

Armenia      Quindío
Armenia    Antioquia
dtype: object

In [55]:
departamento3.values

array(['Quindío', 'Antioquia'], dtype=object)

In [40]:
ciudad

'Armenia'

In [48]:
for x in departamentos_sr['Armenia']:
    print(x)

Quindío
Antioquia


In [45]:
departamentos_sr['Bogotá']

'Cundinamarca'

In [47]:
type(departamentos_sr['Bogotá'])

str

# Saving results

In [ ]:
autores_df.to_csv('autores_colombianos_3.csv', index=False)